In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from codac import *
from collections import deque
import networkx as nx
from scipy.optimize import minimize, rosen, rosen_der

In [5]:
f= open("dgsol-1.3/data/data_set_1/graph_02.txt", mode="r")
li = []
for line in f:
    li.append(line)

In [6]:
f.close()

In [7]:
dico_constrains = {}
dico_interval = {}
for elem in li:
    i = int(elem[6:8].split(" ")[-1])-1
    j = int(elem[16:18].split(" ")[-1])-1
    mini = float(elem[21:42])
    maxi = float(elem[45:-1])
    dico_constrains[(i,j)] = (maxi + mini)/2
    dico_interval[(i,j)] = [mini, maxi]

In [ ]:
dico_interval

In [5]:
links = np.zeros((63,63))
for (i,j) in dico_constrains.keys():
    links[i,j] = 1
    links[j,i] = 1

# Evolution

In [6]:
def c_in(X,dico_interval):
    count = 0
    for key, value in dico_interval.items():
        i,j = key
        mini,maxi = value
        if (np.linalg.norm(X[i]-X[j]) > mini and np.linalg.norm(X[i]-X[j]) < maxi):
            count += 1
    return count

In [7]:
def error(X,dico_interval):
    li = []
    for key, value in dico_interval.items():
        i,j = key
        mini,maxi = value
        li.append([np.linalg.norm(X[i]-X[j]), mini, maxi])
    return li

In [25]:
def cost_indiv(X, dico_interval):
    cout = 0
    for key, value in dico_interval.items():
        i,j = key
        mini, maxi = value
        mean = (maxi+mini)/2
        dist = np.linalg.norm(X[i]-X[j])
        cout += np.max([(maxi**2 -dist**2)/maxi**2, (dist**2 - mini**2)/mini**2, 0])
        #cout += (dist**2  - mean**2)**2 
    return cout

In [15]:
def cost(X,dico_interval):
    n = X.shape[0]
    li_cost = np.zeros(n)
    for i in range(n):
        li_cost[i] = cost_indiv(X[i],dico_interval)
    return li_cost

In [16]:
def select(X,cost, frac = 0.7):
    ordering = np.argsort(cost)
    n = int(frac*ordering.shape[0])
    X_child = X[ordering[:n]]
    return X_child


In [21]:
def variation(X,i):
    atoms = np.random.choice([i for i in range(63)],size=X.shape[0], replace=True)
    X[:,atoms,:] += np.random.normal(loc=0, scale=0.02,size=(X.shape[0],3))
    return X


In [18]:
def remplacement(X,X_child,cost_X, cost_child):
    n = X.shape[0]
    cost_tot = np.concatenate([cost_X,cost_child])
    ordering = np.argsort(cost_tot)[:n]
    X_tot = np.concatenate([X, X_child])
    return X_tot[ordering], cost_tot[ordering]


In [28]:
X = np.random.normal(loc=0, scale= 10,size=(200,63,3))
cost_X = cost(X, dico_interval)
mean = np.mean(cost_X)
N_iter = 1000
n = 0
while n<N_iter:
    if (n%10 == 0):
        print(n)
    X_child =  select(X, cost_X, frac=0.7)
    X_child = variation(X_child,i)
    cost_child = cost(X_child, dico_interval)
    X, cost_X = remplacement(X,X_child, cost_X,cost_child)
    n += 1


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


In [20]:
c_in(X,dico_interval)

0

In [27]:
cost_X[0]

11111.32123564195

In [15]:
li = error(X[0], dico_interval)

In [16]:
li

[[2.0450971529798627, 2.704259074127328, 2.73143824019508],
 [1.275827772024177, 4.472892845799014, 4.51784767228821],
 [0.9529584039082267, 4.193622171822353, 4.235770187344921],
 [1.9188766272758329, 5.678773711814897, 5.735848248515641],
 [3.022312831944629, 2.906097367604874, 2.935305110205748],
 [2.4629885591974556, 2.404425382497866, 2.42859106891218],
 [1.2984190155483035, 4.585412854912853, 4.631498564179853],
 [2.2222292016338114, 2.215912974374222, 2.23818401611664],
 [1.8804480902255505, 1.743909289498742, 1.761436456418456],
 [3.080006498158848, 3.344627737133087, 3.378242930873976],
 [2.964270082395058, 2.977350238047247, 3.007274107892395],
 [2.7707716237720597, 2.354590416611769, 2.378255236512679],
 [1.9864298569204586, 3.260472114587088, 3.293241501924813],
 [1.7849999349139236, 2.897445102154656, 2.926565885128849],
 [3.0361337174422136, 3.019057016685839, 3.049400060667671],
 [1.7689286702885578, 2.327657584353851, 2.351051715722138],
 [2.381604258727069, 3.325718233

# Explore

In [90]:
exist = True
count = 0
for i in range(3,63):
    if not (links[i-3,i] and links[i-2,i] and links[i-1,i]):
        count +=1

# Codac

In [5]:
M = 63

In [ ]:
help(Interval)

In [6]:
li_f = []
li_interval = []
for (l,k), [mini,maxi] in dico_interval.items():
    i = 3*l 
    j = 3*k 
    li_f.append(Function(f"X[{M*3}]", f"sqrt((X[{i}]-X[{j}])^2+(X[{i}+1]-X[{j}+1])^2+(X[{i}+2]-X[{j}+2])^2)"))
    li_interval.append((mini, maxi))



In [7]:
li_ctf = []
X = IntervalVector(M*3,(-10,10))
for (l,k), [mini,maxi] in dico_interval.items():
    i = 3*l 
    j = 3*k
    ctc_f = CtcFunction(Function(f"X[{M*3}]", f"sqrt((X[{i}]-X[{j}])^2+(X[{i}+1]-X[{j}+1])^2+(X[{i}+2]-X[{j}+2])^2)"), Interval(mini,maxi))
    li_ctf.append(ctc_f)
for i in range(M-3):
    ctc_f = CtcFunction(Function(f"X[{M*3}]", f"""sqrt((X[{i}]-X[{i+1}])^2+(X[{i}+1]-X[{i+1}+1])^2+(X[{i}+2]-X[{i+1}+2])^2) * 
                                 sqrt((X[{i+1}]-X[{i+2}])^2+(X[{i}+1]-X[{i+2}+1])^2+(X[{i}+2]-X[{i+2}+2])^2) * 
                                 """), Interval(0,+oo))
    li_ctf.append(ctc_f)
"""for i in range(M-1):
    print(i)
    for j in range(i,M-2):
        if (i!=j):
            for k in range(j,M-3):
                if i!=k and j!=k:
                    ctc_f = CtcFunction(Function(f"X[{M*3}]", f"((X[{i}]-X[{j}])^2+(X[{i}+1]-X[{j}+1])^2+(X[{i}+2]-X[{j}+2])^2)^0.5 - ((X[{i}]-X[{k}])^2+(X[{i}+1]-X[{k}+1])^2+(X[{i}+2]-X[{k}+2])^2)^0.5 - ((X[{k}]-X[{j}])^2+(X[{k}+1]-X[{j}+1])^2+(X[{k}+2]-X[{j}+2])^2)^0.5"), 
                                        Interval(-oo,0))  
                    li_ctf.append(ctc_f)"""
    

'for i in range(M-1):\n    print(i)\n    for j in range(i,M-2):\n        if (i!=j):\n            for k in range(j,M-3):\n                if i!=k and j!=k:\n                    ctc_f = CtcFunction(Function(f"X[{M*3}]", f"((X[{i}]-X[{j}])^2+(X[{i}+1]-X[{j}+1])^2+(X[{i}+2]-X[{j}+2])^2)^0.5 - ((X[{i}]-X[{k}])^2+(X[{i}+1]-X[{k}+1])^2+(X[{i}+2]-X[{k}+2])^2)^0.5 - ((X[{k}]-X[{j}])^2+(X[{k}+1]-X[{j}+1])^2+(X[{k}+2]-X[{j}+2])^2)^0.5"), \n                                        Interval(-oo,0))  \n                    li_ctf.append(ctc_f)'

In [8]:
def contract(X):
    x = X
     
    for (l,k), [mini,maxi] in dico_interval.items():
        i = 3*l 
        j = 3*k 
        a = IntervalVector([x[i],x[i+1],x[i+2]])
        b = IntervalVector([x[j],x[j+1],x[j+2]])
        ctc.dist.contract(a,b,Interval(mini, maxi))
        x[i],x[i+1],x[i+2] = a
        x[j],x[j+1],x[j+2] = b
    return x

In [10]:
def C_in(X, li_f, li_interval):
    if (empty(X)):
        return False
    in_it = True
    for i in range(len(li_f)//2):
        eval = li_f[i].eval(X)
        mini, maxi = li_interval[i]
        if ((eval.lb() < mini) or (eval.ub()>maxi)):
            #in_it = False
            return False
    return True

In [11]:
def C_out(X,li_f, li_interval):
    out_it = False
    for i in range(len(li_f)//2):
        eval = li_f[i].eval(X)
        mini, maxi = li_interval[i]
        if ((eval.lb()> maxi) or (eval.ub() < mini)):
            out_it = True
    return out_it

In [12]:
def get_largest(X):
    max_index = 0
    for i in range(M*3):
        if (X[max_index].diam() < X[i].diam()):
            max_index = i
    return max_index

In [13]:
def empty(X):
    for i in range(M*3):
        if X[i].is_empty():
            return True
    return False

In [180]:
C_in(X,li_f, li_interval)

False

In [201]:
X[0]

[empty]

In [ ]:
def iteration(stack, stack_acc, stack_rej, stack_unc):
    X = stack.pop()
    count +=1
    X = contract(X)
    if not empty(X):
        if (C_in(X,li_f, li_interval)):
            sol = True
            print("sol")
            stack_acc.append(X)
        else:
            if C_out(X, li_f, li_interval):
                stack_rej.append(X)
            else:
                larger_index = get_largest(X)
                if X[larger_index].diam() > tau:
                    X_left, X_right  = X.bisect(larger_index)
                    if not (empty(X_left)):
                        stack.append(X_left)
                    if not (empty(X_right)):
                        stack.append(X_right)
                else:
                    stack_unc.append(X)
    return stack, stack_acc, stack_rej, stack_unc

In [22]:
tau = 0.5
stack = [IntervalVector(M*3,(-10,10))]
stack_acc = deque()
stack_rej = deque()
stack_unc = deque()
sol = False
count = 0
while len(stack) !=0 and (not sol):
    X = stack.pop()
    count +=1
    X = contract(X)
    if not empty(X):
        if (C_in(X,li_f, li_interval)):
            sol = True
            print("sol")
            stack_acc.append(X)
        else:
            if C_out(X, li_f, li_interval):
                stack_rej.append(X)
            else:
                larger_index = get_largest(X)
                if X[larger_index].diam() > tau:
                    X_left, X_right  = X.bisect(larger_index)
                    if not (empty(X_left)):
                        stack.append(X_left)
                    if not (empty(X_right)):
                        stack.append(X_right)
                else:
                    stack_unc.append(X)
    
    


    
        

KeyboardInterrupt: 

In [21]:
X

([5, 10] ; [5, 10] ; [5, 10] ; [6.152885526825072, 9.220571593827709] ; [6.152885526825072, 9.195534771470858] ; [5, 10] ; [5.241076163855894, 8.623583685374904] ; [5.241076163855894, 9.964624863246613] ; [5, 10] ; [3.724294457912892, 6.862147228956447] ; [4.499067388404887, 10] ; [5, 10] ; [4.488631434626367, 9.213198944678585] ; [6.850119104127025, 10] ; [5, 10] ; [3.964775155422134, 6.982387577711068] ; [3.914701510708432, 6.957350755354217] ; [5, 10] ; [6.834939753489637, 9.852552175778572] ; [4.842553651736321, 9.357819360930594] ; [5, 10] ; [4.856233871723165, 10] ; [3.705215805579566, 6.852607902789784] ; [5, 10] ; [6.500750688062066, 9.936831032915452] ; [5.65772465828663, 9.37974499839817] ; [5, 10] ; [6.069785807233515, 10] ; [6.421878062835701, 9.56927016004592] ; [5, 10] ; [4.590703459630224, 10] ; [4.695273286520312, 10] ; [5, 10] ; [4.366649161945668, 10] ; [5.458344020541257, 9.653921633880966] ; [5, 10] ; [6.607780664111488, 10] ; [6.868347701249314, 10] ; [5, 10] ; [5.

In [339]:
solution = stack_acc.pop()

In [342]:
empty(solution)

True

In [328]:
a = stack.pop()
b= a.copy()

In [333]:
a[10].diam()

5.500932611595113

In [335]:
a

([5, 10] ; [5, 10] ; [5, 10] ; [6.152885526825072, 9.220571593827709] ; [6.152885526825072, 10] ; [5, 10] ; [5.241076163855894, 8.623583685374904] ; [5.241076163855894, 10] ; [5, 10] ; [3.724294457912892, 6.862147228956447] ; [4.499067388404887, 10] ; [5, 10] ; [4.488631434626367, 9.213198944678585] ; [6.850119104127025, 10] ; [5, 10] ; [3.964775155422134, 6.982387577711068] ; [6.957350755354216, 10] ; [5, 10] ; [6.834939753489637, 9.852552175778572] ; [4.842553651736321, 9.357819360930594] ; [5, 10] ; [4.856233871723165, 10] ; [3.705215805579566, 6.852607902789784] ; [5, 10] ; [6.500750688062066, 9.936831032915452] ; [5.65772465828663, 9.37974499839817] ; [5, 10] ; [6.069785807233515, 10] ; [6.421878062835701, 9.56927016004592] ; [5, 10] ; [4.590703459630224, 10] ; [4.695273286520312, 10] ; [5, 10] ; [7.183324580972834, 10] ; [5.458344020541257, 9.109479203415136] ; [5, 10] ; [6.607780664111488, 10] ; [3.736695402498628, 6.868347701249315] ; [5, 10] ; [5.819727880505916, 9.31956364526

In [331]:
b

([5, 10] ; [5, 10] ; [5, 10] ; [6.152885526825072, 9.220571593827709] ; [6.152885526825072, 10] ; [5, 10] ; [5.241076163855894, 8.623583685374904] ; [5.241076163855894, 10] ; [5, 10] ; [3.724294457912892, 6.862147228956447] ; [4.499067388404887, 10] ; [5, 10] ; [4.488631434626367, 9.213198944678585] ; [6.850119104127025, 10] ; [5, 10] ; [3.964775155422134, 6.982387577711068] ; [6.957350755354216, 10] ; [5, 10] ; [6.834939753489637, 9.852552175778572] ; [4.842553651736321, 9.357819360930594] ; [5, 10] ; [4.856233871723165, 10] ; [3.705215805579566, 6.852607902789784] ; [5, 10] ; [6.500750688062066, 9.936831032915452] ; [5.65772465828663, 9.37974499839817] ; [5, 10] ; [6.069785807233515, 10] ; [6.421878062835701, 9.56927016004592] ; [5, 10] ; [4.590703459630224, 10] ; [4.695273286520312, 10] ; [5, 10] ; [7.183324580972834, 10] ; [5.458344020541257, 9.109479203415136] ; [5, 10] ; [6.607780664111488, 10] ; [3.736695402498628, 6.868347701249315] ; [5, 10] ; [5.819727880505916, 9.31956364526

In [89]:
# Create a 3D scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Extract x, y, and z coordinates
x, y, z = best.T

# Plot the atoms
ax.scatter(x, y, z, c='b', marker='o', s=100)

# Set labels for the axes
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Plot the lines between connected atoms
for link in dico_constrains:
    print(link)
    x_link, y_link, z_link = best[list(link)].T
    mini, maxi = dico_interval[link]

    color = (mini < ((x_link[0] - x_link[1])**2 + (y_link[0] - y_link[1])**2 + (z_link[0] - z_link[1])**2)**.5 < maxi )
    print(((x_link[0] - x_link[1])**2 + (y_link[0] - y_link[1])**2 + (z_link[0] - z_link[1])**2)**.5, mini, maxi, color)
    if color:
        ax.plot(x_link, y_link, z_link, c='g')
    else:
        ax.plot(x_link, y_link, z_link, c='r')



# Show the 3D plot
plt.show()

Help on method bisect in module codac.core:

bisect(...) method of codac.core.IntervalVector instance
    bisect(self: codac.core.IntervalVector, i: int, ratio: float = 0.5) -> Tuple[codac.core.IntervalVector, codac.core.IntervalVector]
    
    Bisect the box
    The box is bisected along the dimension \a i
    and with a ratio \a ratio. If (self)[i] is the interval [a,a+d]:
    
        + The first box of the result is (self)[0]x...x(self)[i-1]x[a+ratio*d]x...
        + The second box is (self)[0]x...x(self)[i-1]x[a+ratio*d,a+d]x...
    
    Args:
        i (int) : dimension to besect along
        radio (float) : ratio Default  is 0.5. Need to belong to [0, 1]
    
    Return:
        list<IntervalVector> : two sub intervalvectors.



# Lsbuild

In [8]:
G = nx.Graph()

In [9]:
G.add_nodes_from([i for i in range(63)])

In [10]:
for (i,j), (mini, maxi) in dico_interval.items():
    G.add_edges_from([(i,j,{"mini":mini, "maxi":maxi})])

In [11]:
cliques = nx.find_cliques(G)

In [12]:
maxi_clique = max(nx.find_cliques(G), key=len)

In [13]:
sub = nx.Graph()
for node in maxi_clique:
    sub.add_node(node)

In [14]:
for (i,j), (mini, maxi) in dico_interval.items():
    if i in maxi_clique and j in maxi_clique:
        sub.add_edges_from([(i,j,{"mini":mini, "maxi":maxi})])

In [15]:
list(sub.nodes)

[18, 20, 21, 19, 22, 24, 25, 26, 23]

In [16]:
def get_edm(nodes,t, graph):
    n = len(nodes)
    matrix = np.zeros((n,n))
    for k in range(n-1):
        for l in range(k+1,n):
            i = nodes[k]
            j = nodes[l]
            low = graph.get_edge_data(i,j)["mini"]
            up = graph.get_edge_data(i,j)["maxi"]
            matrix[k,l] = (1-t[k,l])*low + t[k,l]*up
            matrix[l,k] = (1-t[k,l])*low + t[k,l]*up
    return matrix

In [17]:
def get_x(EDM):
    U, S, Vh = np.linalg.svd(D)
    x = U[:,:3]
    return x

In [18]:
lamb = 1
tau = 0.1

def theta(x,i,j,c):
    return (lamb**2 *(c-np.sqrt(np.linalg.norm(x[3*i:3*(i+1)]-x[3*j:3*(j+1)])**2 + tau**2))**2 + tau**2)
def phi(x,i,j, low,up):
    value = lamb*(low - up) + theta(x,i,j,low) + theta(x,i,j,up)
    return value
def cost(x,nodes,graph):
    n = len(nodes)
    cost_value = 0
    for k in range(n-1):
        for l in range(k+1,n):
            i = nodes[k]
            j = nodes[l]
            low = graph.get_edge_data(i,j)["mini"]
            up = graph.get_edge_data(i,j)["maxi"]
            cost_value += phi(x,k,l,low,up)
    return cost_value


In [19]:
def get_n_error(X,nodes, graph):
    n = len(nodes)
    cost_value = 0
    count = 0
    relative_error = []
    count_tot = 0
    for k in range(n-1):
        for l in range(k+1,n):
            i = nodes[k]
            j = nodes[l]
            count_tot += 1
            low = graph.get_edge_data(i,j)["mini"]
            up = graph.get_edge_data(i,j)["maxi"]
            dist = np.linalg.norm(X[k]-X[l])
            print(f"edge: ({i,j}")
            print(dist)
            print(f"low : {low}    up : {up}")
            print()
            if (dist>up or dist < low):
                count += 1
                relative_error.append(np.min([np.abs(dist-low)/low,np.abs(dist-up)/up]))

    return

In [20]:
t = np.ones((9,9)) * 0.7
D = get_edm(list(sub.nodes),t,sub)
x = get_x(D)
res = minimize(cost,x.flatten(),args=(list(sub.nodes),sub),method='BFGS')
X = res.x.reshape(9,3)

In [ ]:
get_n_error(X,list(sub.nodes),sub)

In [21]:
dico_index_to_X = {}
dico_X_to_index = {}
count = 0
for node in list(sub.nodes):
    dico_index_to_X[node] = count
    count += 1
count = 0
for node in list(sub.nodes):
    dico_X_to_index[count] = node
    count += 1 

In [22]:
dico_index_to_X

{18: 0, 20: 1, 21: 2, 19: 3, 22: 4, 24: 5, 25: 6, 26: 7, 23: 8}

In [23]:
def best_node(nodes_base,node_to_choose, graph):
    counter = [[] for _ in range(len(node_to_choose))]
    for (i,j) in graph.edges:
        if ((i in nodes_base) and (j not in nodes_base)):
            counter[j].append(i)
        if ((j in nodes_base) and (i not in nodes_base)):
            counter[i].append(j)
    for i in range(len(counter)):
        if (len(counter[i])>=3):
            return i, counter[i]


In [24]:
node, node_base_connected = best_node(list(sub.nodes), list(G.nodes), G)

In [227]:
X

array([[ 0.92386473, -3.57328437, -0.09970757],
       [ 1.02019631, -1.0945451 ,  0.45212424],
       [ 0.52666911, -1.48547055, -1.18063096],
       [-0.94897128, -2.13482179,  1.38250944],
       [-1.21339056,  0.10897043, -1.18954116],
       [ 0.424028  ,  1.58876684,  0.84492453],
       [-1.32911259,  2.28802765, -1.21433089],
       [-1.0108888 ,  4.1059399 ,  0.04734208],
       [-1.35798251,  0.11808582,  0.55468038]])

In [25]:
def update_EDM(D, X, nodes_base, dico_index_to_X):
    n = len(nodes_base)
    for k in range(n-1):
        for l in range(k+1, n):
            i = dico_index_to_X[nodes_base[k]]
            j = dico_index_to_X[nodes_base[l]]
            D[i,j] = np.linalg.norm(X[i] - X[j])
            D[j,i] = np.linalg.norm(X[i] - X[j])
    return D

In [26]:
def get_d(i,j,graph,t):
    low = graph.get_edge_data(i,j)["mini"]
    up = graph.get_edge_data(i,j)["maxi"]
    return (1-t)*low + t*up

In [27]:
def solve(node_base_connected, node_to_solve, graph,X, t=0.7):
    A = []
    B = []
    for (i,j) in graph.edges:
        if (i == node_to_solve and j in node_base_connected[1:]):  
            A.append(X[dico_index_to_X[j]])
            B.append((get_d(j,node_base_connected[0],graph,t)**2 - get_d(j,i,graph,t) + get_d(i,node_base_connected[0], graph,t) ))
        if (j == node_to_solve and i in node_base_connected[:1]):
            A.append(X[dico_index_to_X[i]])
            B.append((get_d(j,node_base_connected[0],graph,t)**2 - get_d(j,i,graph,t) + get_d(i,node_base_connected[0], graph,t)))
    return np.linalg.lstsq(A,B), np.array(A), np.array(B)


In [114]:
sub.nodes

NodeView((18, 20, 21, 19, 22, 24, 25, 26, 23))

In [28]:
links = {}
for i in range(63):
    links[i] = 0
for (i,j) in G.edges:
    if (i in list(sub.nodes)):
        links[j] += 1
    if (j in list(sub.nodes)):
        links[i] += 1


In [29]:
print(G.get_edge_data(10,18))
print(G.get_edge_data(10,20))
print(G.get_edge_data(10,21))

{'mini': 3.280604499173894, 'maxi': 3.346886033315147}
{'mini': 3.185788056352778, 'maxi': 3.250153913278571}
{'mini': 2.469650991537058, 'maxi': 2.519547971363112}


In [40]:
X[2][2]

-1.1806063960678272

In [ ]:
links

In [89]:
(sol , residuals, rank, s), A, B = solve(list(node_base_connected),node, G, X)

C:\Users\thoma\AppData\Local\Temp\ipykernel_9684\926669581.py:11: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  return np.linalg.lstsq(A,B), np.array(A), np.array(B)


In [90]:
sol

array([ 2.64255451, -3.5437269 ,  0.08260722])

In [86]:
X_new = np.concatenate([X,sol.reshape(1,3)])
dico_X_to_index[10] = 10
dico_index_to_X[10] = 10

In [87]:
sub.add_node(10)

In [82]:
B.shape

(2,)

In [80]:
np.linalg.norm(A@sol - B)

1.9860273225978185e-15

In [ ]:
for edge in G.edges:
    print(edge)

In [54]:
X

array([[ 0.9237939 , -3.57316066, -0.09976981],
       [ 1.02014498, -1.09452797,  0.45209783],
       [ 0.52663748, -1.48539666, -1.1806064 ],
       [-0.948962  , -2.13476689,  1.38241831],
       [-1.21334067,  0.10900231, -1.18948551],
       [ 0.42402484,  1.58868038,  0.84494258],
       [-1.32903637,  2.28797949, -1.21422512],
       [-1.0108103 ,  4.10579158,  0.04744276],
       [-1.35793261,  0.11807908,  0.55467022]])

In [174]:
sub.get_edge_data(26,23)

{'mini': 4.015538909038237, 'maxi': 4.055897098053647}

In [162]:
sub.nodes

NodeView((18, 20, 21, 19, 22, 24, 25, 26, 23))

In [ ]:
def